<a href="https://colab.research.google.com/github/ShubhranshuPattnaik/Custom-Sentiment-Analysis-with-DistilBERT/blob/main/Custom_Sentiment_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import pandas as pd
import tensorflow as tf
from transformers import DistilBertTokenizerFast, TFDistilBertForSequenceClassification
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix

In [4]:
df = pd.read_csv('/content/SMSSpamCollection.txt', sep='\t', names=["label", "message"])
X = list(df['message'])
y = list(df['label'])
y = list(pd.get_dummies(y, drop_first=True)['spam'])

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=0)

In [5]:
!pip install transformers

In [6]:
from transformers import DistilBertTokenizerFast, TFDistilBertForSequenceClassification

tokenizer = DistilBertTokenizerFast.from_pretrained('distilbert-base-uncased')
train_encodings = tokenizer(X_train, truncation=True, padding=True, return_tensors='tf')
test_encodings = tokenizer(X_test, truncation=True, padding=True, return_tensors='tf')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

In [7]:
y_train = tf.convert_to_tensor(y_train)
y_test = tf.convert_to_tensor(y_test)

In [8]:
train_dataset = tf.data.Dataset.from_tensor_slices((dict(train_encodings), y_train)).batch(8)
test_dataset = tf.data.Dataset.from_tensor_slices((dict(test_encodings), y_test)).batch(16)

In [9]:
model = TFDistilBertForSequenceClassification.from_pretrained("distilbert-base-uncased")
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=5e-5),
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=[tf.keras.metrics.SparseCategoricalAccuracy()])

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFDistilBertForSequenceClassification: ['vocab_transform.weight', 'vocab_projector.bias', 'vocab_layer_norm.bias', 'vocab_transform.bias', 'vocab_layer_norm.weight']
- This IS expected if you are initializing TFDistilBertForSequenceClassification from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFDistilBertForSequenceClassification from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
Some weights or buffers of the TF 2.0 model TFDistilBertForSequenceClassification were not initialized from the PyTorch model and are newly initialized: ['pre_classifier.weight', 'pre_classifier.bias', 'classifier.weight', 'classifier.bias']
You should 

In [ ]:
model.fit(train_dataset, epochs=2, batch_size=8)

Epoch 1/2
 29/558 [>.............................] - ETA: 1:49:45 - loss: 0.2365 - sparse_categorical_accuracy: 0.9310

In [ ]:
loss, accuracy = model.evaluate(test_dataset)
print(f'Loss: {loss}')
print(f'Accuracy: {accuracy}')

In [ ]:
predictions = model.predict(test_dataset).logits
predicted_classes = tf.argmax(predictions, axis=1)

In [ ]:
cm = confusion_matrix(y_test, predicted_classes)
print(cm)

In [ ]:

model.save_pretrained('senti_model')

In [ ]:
!pip install gradio

In [ ]:
import gradio as gr
def predict(text):
    encodings = tokenizer(text, truncation=True, padding=True, return_tensors='tf')
    predictions = model(encodings).logits
    predicted_class = tf.argmax(predictions, axis=1).numpy()[0]
    return "Spam" if predicted_class == 1 else "Not Spam"


interface = gr.Interface(fn=predict, inputs="text", outputs="text")


interface.launch()